This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([1.50371026e+02, 3.25497888e+02, 2.00268881e+02, 2.22927321e+02,
       1.82129033e+02, 2.01402113e+02, 3.46912901e+02, 1.21202059e+02,
       1.10462825e+02, 1.20156284e+02, 1.06566685e+02, 1.86835775e+02,
       2.00047469e+02, 1.45936466e+02, 1.47491884e+02, 1.30710796e+02,
       9.93875156e+01, 8.91149867e+01, 1.44746721e+02, 1.91511615e+02,
       1.70828354e+02, 1.96095876e+02, 1.42483393e+02, 8.44124962e+01,
       5.96792027e+01, 1.87427677e+02, 1.56941241e+02, 2.89476692e+01,
       2.41842340e+01, 9.78662706e+01, 8.73132966e+01, 2.93743989e+02,
       1.08666855e+02, 4.17751932e+02, 2.54265050e+02, 7.30199956e+01,
       1.06616452e+02, 7.19005615e+01, 1.57715585e+02, 1.78235739e+02,
       1.75379626e+02, 2.74349274e+02, 1.03801111e+02, 2.78307782e+02,
       1.28364345e+02, 2.38334229e+02, 8.21644400e+00, 6.33627358e+01,
       0.00000000e+00, 1.27681884e+02, 1.79258723e+02, 9.04068336e+01,
       3.04032713e+01, 6.78471000e-01, 3.65282198e+01, 7.48766194e+00,
       5.66738367e+01, 1.29458568e+01, 5.42920320e+01, 1.01797990e+02,
       1.28576453e+02, 5.35777138e+01, 0.00000000e+00, 6.61459738e+01,
       1.40134457e+02, 9.68586984e+01, 2.36699859e+02, 5.04901919e+01,
       1.49850095e+02, 2.06158236e+02, 1.45197832e+02, 2.51771744e+02,
       3.77502009e+02, 1.19774983e+02, 1.17650530e+02, 5.95521104e+01,
       4.52159618e+01, 6.01231126e+01, 0.00000000e+00, 2.79926398e+01,
       0.00000000e+00, 2.03972667e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.45062097e+01,
       2.53695907e+01, 1.54429064e+01, 1.35168013e+02, 1.18479708e+01,
       2.63184751e+01, 1.08224668e+02, 4.06187327e+01, 6.70851015e+01,
       4.33637207e+01, 4.02544398e+01, 3.65075124e+01, 4.89943291e+01,
       4.98477351e+01, 3.13433527e+01, 1.78001455e+02, 1.57374363e+02,
       1.81910775e+02, 1.50508186e+02, 1.14247314e+02, 9.57636772e+01,
       1.15200294e+02, 0.00000000e+00, 1.00457045e+02, 3.04689791e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.05625634e+00,
       9.52352009e+00, 5.26296357e+01, 3.41493981e+01, 3.55034057e+01,
       4.07973166e+01, 4.23248061e+01, 0.00000000e+00, 7.47468692e+01,
       7.96465197e+01, 4.54332140e+01, 3.54620958e+01, 3.51232216e+01,
       3.51583952e+01, 0.00000000e+00, 7.33952221e+01, 7.30016409e+01,
       1.19114168e+02, 1.07975067e+02, 1.24630840e+02, 1.12573845e+02,
       7.58854939e+01, 0.00000000e+00, 9.85807552e+01, 0.00000000e+00,
       3.50982856e+00, 1.29323672e+02, 2.78984665e+02, 2.83906453e+02,
       4.68034609e+02, 5.24822446e+01, 6.99723855e+01, 1.22311356e+02,
       0.00000000e+00, 1.71956703e+02, 1.58427043e+02, 2.57075963e+02,
       2.84898190e+02, 1.39345081e+02, 3.67624432e+02, 0.00000000e+00,
       4.78013760e+01, 0.00000000e+00, 0.00000000e+00, 1.00440195e+02,
       0.00000000e+00, 5.44675418e+01, 0.00000000e+00, 0.00000000e+00,
       5.12388154e+01, 5.85465181e+01, 4.55261914e+01, 0.00000000e+00,
       0.00000000e+00, 5.09486685e+01, 4.45494170e+01, 2.34800709e+02,
       1.32773863e+02, 7.54658121e+01, 7.83502295e+01, 7.67577252e+01,
       6.94770317e+01, 8.29266298e+01, 6.06316356e+01, 4.42032278e+01,
       3.32733667e+01, 5.67330262e+01, 0.00000000e+00, 6.08628852e+01,
       3.96932977e+01, 4.28158804e+01, 4.52879705e+01, 1.38758173e+02,
       0.00000000e+00, 9.38711282e+01, 0.00000000e+00, 9.13449975e+01,
       6.59849632e+01, 1.41373726e+02, 3.61816245e+01, 2.64912691e+01,
       6.33172566e+01, 0.00000000e+00, 2.57130076e+01, 8.22953902e+01,
       1.10716943e+02, 9.52268434e+01, 5.48277307e+01, 5.29102783e+01,
       1.37881939e+02, 8.06894184e+01, 3.57280662e+01, 2.32281891e+01,
       1.89735033e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.56564178e+00, 4.45467226e+01, 1.03743969e+02,
       1.33030325e+02, 2.44348237e+02, 1.68517827e+02, 1.85197855e+02,
       2.17252824e+02, 2.09019845e+02, 2.15604850e+02, 1.83828681e+02,
       1.65118221e+02, 1.58037965e+02, 2.13314660e+02, 3.42040019e+02,
       2.52767162e+02, 2.49724729e+02, 1.96074159e+02, 1.36293767e+02,
       1.89952208e+02, 2.20003381e+02, 1.39013581e+02, 1.63799175e+02,
       2.85344428e+02, 9.81125625e+01, 1.33954726e+02, 8.30462436e+01,
       2.05092926e+02, 2.31128112e+02, 1.54419624e+02, 2.92536084e+02,
       2.22246511e+02, 2.37992715e+02, 3.09856621e+02, 3.20529825e+02,
       3.49453381e+02, 2.32699819e+02, 1.99002346e+02, 2.05564030e+02,
       7.08220872e+01, 1.35164757e+02, 2.47684975e+02, 1.51356144e+02,
       2.47005095e+02, 1.03456135e+02, 1.54641811e+02, 1.92088747e+02,
       1.40265182e+02, 1.92010418e+02, 9.54547321e+01, 1.08425980e+02,
       1.97309000e+02, 2.16454613e+02, 3.13722881e+02, 2.59086381e+02,
       3.49605405e+02, 3.05414508e+02, 7.89063338e+01, 1.44154498e+02,
       6.56442278e+01, 1.70005981e+02, 1.09381016e+02, 2.44800345e+02,
       1.62784213e+02, 1.51898311e+02, 6.02850907e+01, 1.39231351e+02,
       1.50801863e+02, 1.65456460e+02, 8.31507296e+01, 8.68301581e+01,
       2.38535058e+02, 1.78519868e+02, 1.44135045e+02, 2.28169699e+02,
       8.57231665e+02, 1.61637650e+02, 1.81019902e+02, 1.20158300e+02,
       1.01883876e+02, 9.27064680e+01, 1.41354043e+02, 1.10065846e+02,
       1.90046405e+02, 1.09913097e+02, 9.27983292e+01, 8.32175656e+01,
       1.25312626e+02, 6.28478449e+01, 1.17389412e+02, 5.18182329e+01,
       3.70668895e+01, 1.13321950e+02, 4.43281812e+01, 4.34380611e+01,
       4.18854297e+01, 4.30159281e+01, 7.21745594e+01, 5.66837962e+01,
       4.85930390e+01, 4.15835904e+01, 5.12762345e+01, 1.61563184e+02,
       1.76433607e+02, 1.49540965e+02, 1.47336842e+02, 1.78369049e+02,
       1.98013501e+02, 2.10616630e+02, 1.46552154e+02, 1.33312148e+02,
       1.59787224e+02, 1.47483468e+02, 1.71603969e+02, 1.37406686e+02,
       3.07711248e+01, 1.00815664e+02, 1.05722180e+02, 1.35954460e+02,
       1.85051275e+02, 1.71490363e+02, 2.08035658e+02, 2.91099073e+02,
       1.15594393e+02, 2.16122195e+02, 1.41965660e+02, 1.35369317e+02,
       2.44717542e+02, 8.98167378e+01, 9.12669747e+01, 8.28791006e+01,
       7.98648319e+01, 6.69909409e+01, 1.83475519e+02, 7.10907636e+01,
       8.71299483e+01, 1.51299348e+01, 5.97413320e+01, 7.29422616e+01,
       3.97988019e+01, 5.91429461e+01, 1.65733079e+02, 1.98665047e+02,
       7.14796192e+01, 4.97936119e+01, 7.84670662e+01, 4.06890701e+01,
       3.50459396e+01, 1.07860026e+02, 1.11369551e+02, 0.00000000e+00,
       1.01020674e+02, 1.10110223e+02, 8.45812329e+01, 9.04350450e+01,
       1.55909864e+02, 0.00000000e+00, 2.15058665e+01, 1.78464077e+01,
       1.36252990e+02, 0.00000000e+00, 7.26725204e+01, 1.00472922e+02,
       3.42214622e+01, 0.00000000e+00, 2.12813897e+01, 2.63539598e+01,
       2.15324072e+01, 3.73769471e+01, 8.47289652e+01, 2.77992144e+01,
       5.81498187e+01, 2.01387280e+01, 1.39062159e+02, 7.26501274e+01,
       2.21768007e+02, 6.74148765e+01, 1.16752352e+02, 7.04541654e+01,
       7.05986967e+01, 6.55202594e+01, 0.00000000e+00, 4.50377785e+01,
       3.81742435e+01, 3.83815528e+01, 0.00000000e+00, 8.61157898e+01,
       7.56793957e+01, 1.57611747e+02, 5.33006835e+01, 3.76093386e+01,
       0.00000000e+00, 0.00000000e+00, 1.26616164e+02, 1.20865840e+02,
       4.59371875e+01, 2.07374692e+01, 1.60395108e+02, 4.26966290e+01,
       2.84167104e+01, 6.65012486e+01, 0.00000000e+00, 3.87785474e+01,
       5.00275181e+01, 3.34624193e+01, 5.43341092e+01, 9.90236861e+01,
       3.30048005e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.94101993e+01])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 3.15225147e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 1.18364042e+001, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 6.11556778e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       2.12969881e-200, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       5.88989627e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 5.40679990e-199, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       1.47151242e-316, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 4.49083689e+001, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_7/test_residuals_histogram.png'
stats = OrderedDict([('R2', 2.321666962123992e-06), ('root_mean_squared_error', 141.99365299105017), ('mean_absolute_error', 107.19722686075865), ('rmse_over_stdev', 1.4794282669999237)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
